In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:3]:
        print(os.path.join(dirname, filename))
    if len(filenames) > 3:
        print("...")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/captcha-hacker/sample_submission.csv
/kaggle/input/captcha-hacker/test/task1/x4LPcV5n6IvLj4vz.png
/kaggle/input/captcha-hacker/test/task1/W88fVMlAs5IpsXn4.png
/kaggle/input/captcha-hacker/test/task1/ZWDL6pUMfPu5c9jh.png
...
/kaggle/input/captcha-hacker/test/task2/jMalnsI5a5IWxYAi.png
/kaggle/input/captcha-hacker/test/task2/ihE9HHgyOINGEMcO.png
/kaggle/input/captcha-hacker/test/task2/ZATEVW3P5s0akZjd.png
...
/kaggle/input/captcha-hacker/test/task3/cXBlxYfvQWbiK7dn.png
/kaggle/input/captcha-hacker/test/task3/5gEp1jR9jNNfuqlk.png
/kaggle/input/captcha-hacker/test/task3/hEQ0WQtB9B7j8C2f.png
...
/kaggle/input/captcha-hacker/train/annotations.csv
/kaggle/input/captcha-hacker/train/task1/H85RQ6dbWUvLSIDV.png
/kaggle/input/captcha-hacker/train/task1/n2GC8uY1N4QfvVxe.png
/kaggle/input/captcha-hacker/train/task1/XOqfRx2R6SnoEjFr.png
...
/kaggle/input/captcha-hacker/train/task2/Mr4B2zxXk92hyzn9.png
/kaggle/input/captcha-hacker/train/task2/SIuRCnlK8VS91FhX.png
/kaggle/input/captcha-h

In [ ]:
import csv
import cv2
import numpy as np
import random
import os

from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models

In [ ]:
TRAIN_PATH = "/kaggle/input/captcha-hacker/train"
TEST_PATH = "/kaggle/input/captcha-hacker/test"
device = "cuda"
# try device = "cuda" 
# and change your settings/accelerator to GPU if you want it to run faster

In [ ]:
alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
result = ['0','1','2','3','4','5','6','7','8','9','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']

In [ ]:
class Task1Dataset(Dataset):
    def __init__(self, data, root, return_filename=False):
        self.data = [sample for sample in data if sample[0].startswith("task1")]
        self.return_filename = return_filename
        self.root = root
    
    def __getitem__(self, index):
        filename, label = self.data[index]
        img = cv2.imread(f"{self.root}/{filename}")
        img = cv2.resize(img, (224, 224))
#         img = np.mean(img, axis=2)
#         print(img.shape)

        if self.return_filename:
            return torch.FloatTensor(img / 256), filename
        else:
            return torch.FloatTensor(img / 256), int(label)

    def __len__(self):
        return len(self.data)

In [ ]:
class Task2Dataset(Dataset):
    def __init__(self, data, root, return_filename=False):
        self.data = [sample for sample in data if sample[0].startswith("task2")]
        self.return_filename = return_filename
        self.root = root
    
    def __getitem__(self, index):
        filename, label = self.data[index]
        
        labels = [0 for i in range(72)]

        if label[0].isalpha(): 
            idx = alphabet.index(label[0]) + 10
            labels[idx] = 1
        else:
            labels[int(label[0])] = 1

        if len(label) > 1:
            if label[1].isalpha():
                idx = alphabet.index(label[1]) + 10
                labels[idx + 36] = 1
            else:
                labels[int(label[1]) + 36] = 1
                 
        img = cv2.imread(f"{self.root}/{filename}")
        img = cv2.resize(img, (224, 224))
        
        if self.return_filename:
            return torch.FloatTensor(img / 256), filename
        else:
                return torch.FloatTensor(img / 256), torch.tensor(labels)

    def __len__(self):
        return len(self.data)

In [ ]:
class Task3Dataset(Dataset):
    def __init__(self, data, root, return_filename=False):
        self.data = [sample for sample in data if sample[0].startswith("task3")]
        self.return_filename = return_filename
        self.root = root
    
    def __getitem__(self, index):
        filename, label = self.data[index]
        
        labels = [0 for i in range(144)]
        
        if label[0].isalpha(): 
            idx = alphabet.index(label[0]) + 10
            labels[idx] = 1
        else:
            labels[int(label[0])] = 1

        if len(label) > 1:
            if label[1].isalpha():
                idx = alphabet.index(label[1]) + 10
                labels[idx + 36] = 1
            else:
                labels[int(label[1]) + 36] = 1
    
        if len(label) > 2:
            if label[2].isalpha():
                idx = alphabet.index(label[2]) + 10
                labels[idx + 72] = 1
            else:
                labels[int(label[2]) + 72] = 1
            
        if len(label) > 3:
            if label[3].isalpha():
                idx = alphabet.index(label[3]) + 10
                labels[idx + 108] = 1
            else:
                labels[int(label[3]) + 108] = 1
            
        img = cv2.imread(f"{self.root}/{filename}")
        img = cv2.resize(img, (480, 360))
        
        if self.return_filename:
            return torch.FloatTensor(img / 256), filename
        else:
                return torch.FloatTensor(img / 256), torch.tensor(labels)

    def __len__(self):
        return len(self.data)

In [ ]:
class Model1(nn.Module):
    def __init__(self):
#         super().__init__()
#         self.layers = nn.Sequential(
#             nn.Linear(1024, 512),
#             nn.LeakyReLU(),
#             nn.Linear(512, 10)
#         )

        super().__init__()
        # 載入 ResNet18 類神經網路結構
        self.model = models.resnet18(pretrained=True)
        # 修改輸出層輸出數量
        self.model.fc = nn.Linear(512, 10)
        
        
    def forward(self, x):
#         b, h, w = x.shape
#         x = x.view(b, h*w)
#         return self.layers(x)
        logits = self.model(x)
        return logits

In [ ]:
class Model2(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(512, 72)  # 72
        
    def forward(self, x):
        logits = self.model(x)
        return logits

In [ ]:
class Model3(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(512, 144)  # 144
        
    def forward(self, x):
        logits = self.model(x)
        return logits

In [ ]:
model_1 = Model1().to(device)
model_1.load_state_dict(torch.load('save1.pt'))

model_2 = Model2().to(device)
model_2.load_state_dict(torch.load('save2.pt'))

model_3 = Model3().to(device)
model_3.load_state_dict(torch.load('save3.pt'))

<All keys matched successfully>

In [ ]:
test_data = []
    
with open(f'{TEST_PATH}/../sample_submission.csv', newline='') as csvfile:
#     cnt = sum(1 for line in csvfile)
#     print(cnt)
    for row in csv.reader(csvfile, delimiter=','):
        test_data.append(row)

# task1        
test1_ds = Task1Dataset(test_data, root=TEST_PATH, return_filename=True)
test1_dl = DataLoader(test1_ds, batch_size=64, num_workers=4, drop_last=False, shuffle=False)
# task2
test2_ds = Task2Dataset(test_data, root=TEST_PATH, return_filename=True)
test2_dl = DataLoader(test2_ds, batch_size=64, num_workers=4, drop_last=False, shuffle=False)
# task 3
test3_ds = Task3Dataset(test_data, root=TEST_PATH, return_filename=True)
test3_dl = DataLoader(test3_ds, batch_size=64, num_workers=4, drop_last=False, shuffle=False)


file = open('submission.csv', 'w', newline='')
csv_writer = csv.writer(file)
csv_writer.writerow(["filename", "label"])

cnt = 0
model_1.eval()
for image, filenames in test1_dl:
    image = image.to(device)
    image = image.permute(0,3,1,2)
    
    pred = model_1(image)
    pred = torch.argmax(pred, dim=1)

    for i in range(len(filenames)):
        csv_writer.writerow([filenames[i], str(pred[i].item())])
        cnt += 1
print(cnt)

model_2.eval()
for image, filenames in test2_dl:
    image = image.to(device)
    image = image.permute(0,3,1,2)
    
    pred = model_2(image)
    pred1, pred2 = torch.split(pred, 36, dim=1)
    pred1 = torch.argmax(pred1, dim=1)
    pred2 = torch.argmax(pred2, dim=1)
    pred1 = [result[i] for i in pred1]
    pred2 = [result[i] for i in pred2]
    
    for i in range(len(filenames)):
        csv_writer.writerow([filenames[i], pred1[i]+pred2[i]])
        cnt += 1
print(cnt)

model_3.eval()
for image, filenames in test3_dl:
    image = image.to(device)
    image = image.permute(0,3,1,2)
    
    pred = model_3(image)
    pred1, pred2, pred3, pred4 = torch.split(pred, 36, dim=1)
    pred1 = torch.argmax(pred1, dim=1)
    pred2 = torch.argmax(pred2, dim=1)
    pred3 = torch.argmax(pred3, dim=1)
    pred4 = torch.argmax(pred4, dim=1)
    pred1 = [result[i] for i in pred1]
    pred2 = [result[i] for i in pred2]
    pred3 = [result[i] for i in pred3]
    pred4 = [result[i] for i in pred4]
    
    for i in range(len(filenames)):
        csv_writer.writerow([filenames[i], pred1[i] + pred2[i] + pred3[i] + pred4[i]])
        cnt += 1    
print(cnt)

file.close()

6500
9000
10000
